In [1]:
from ipyparallel import Client
import numpy as np
from functools import reduce
import operator

In [2]:
%%bash
scancel -u battila93

In [3]:
N=2

In [4]:
%%bash
sbatch -p hpc2005 -N 2 -B 1:2:1 run_ipengines.sh

Submitted batch job 1607024


In [8]:
%%bash
squeue -u battila93

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1607024   hpc2005 run_ipen battila9  R       0:14      2 blade[07-08]


In [9]:
profile = open("slurm_profile").read().replace("\n","")

c = Client(profile=profile)
dv = c[:]

In [10]:
dv.queue_status()

{'unassigned': 0,
 0: {'completed': 0, 'queue': 0, 'tasks': 0},
 1: {'completed': 0, 'queue': 0, 'tasks': 0}}

In [11]:
def calcint(alphas):
    import numpy as np
    from KGInt import integrate
    rs = np.linspace(2, 30., 2, np.double)
    errors = np.array([1e-9]*2, np.double)
    error_rngs = np.array([1e-9]*2, np.double)
    results = []
    for alpha in alphas:
        results.append(integrate([alpha]*len(rs), rs, errors, error_rngs))
    return results

In [12]:
alphas = np.linspace(0.5, 1.99, N*2)

In [13]:
cr = dv.map_async(calcint, list(zip(*[iter(alphas)]*N)) )

In [14]:
cr.completed

[datetime.datetime(2017, 4, 30, 13, 43, 58, 38517, tzinfo=tzlocal()),
 datetime.datetime(2017, 4, 30, 13, 43, 58, 130553, tzinfo=tzlocal())]

In [15]:
cr.stdout

['->CPU: elapsed time = 108.0 ms\n->CPU: elapsed time = 154.0 ms\n',
 '->CPU: elapsed time = 170.0 ms\n->CPU: elapsed time = 181.0 ms\n']

In [16]:
result = reduce(operator.concat, cr.get())

In [17]:
result

[array([[  2.10883111e-02,   5.85118164e-11],
        [  1.56770888e-02,   6.19125307e-11]]),
 array([[  1.75119376e-02,   2.94106926e-11],
        [  1.45623132e-02,   3.10965486e-11]]),
 array([[  1.28546781e-02,   1.97221009e-11],
        [  1.28337757e-02,   2.07229801e-11]]),
 array([[  8.34109004e-03,   1.48502951e-11],
        [  1.06624595e-02,   1.53603794e-11]])]

In [18]:
%%bash

scancel -u battila93